In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
!pip install contractions
import contractions
from sklearn.model_selection import train_test_split

# For Preprocessing
import spacy
import string

# Load the data
# df = pd.read_csv('fake_job_postings.csv')

# Load data from colab
from google.colab import drive
drive.mount('/content/drive')

!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 16.1 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('/content/drive/My Drive/BT4012 Group 06/Data/feature_engineering_data_v3.csv')

time: 6.02 s (started: 2023-10-25 03:43:40 +00:00)


In [ ]:
pd.set_option('display.max_columns', None)
df.head()

,job_id,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,min_salary,max_salary,tokenized_title,pos_tags_title,tokenized_company_profile,pos_tags_company_profile,tokenized_description,pos_tags_description,tokenized_requirements,pos_tags_requirements,tokenized_benefits,pos_tags_benefits,country,province,city,sentiment score_profile,sentiment score_requirement,sentiment score_benefits,sentiment score_description,embeddings_profile,department_frequency,department_mean_target,mean_embeddings_description,num_of_nouns_company_profile,nouns_vs_words_company_profile,nouns_vs_length_company_profile,num_of_nouns_description,nouns_vs_words_description,nouns_vs_length_description,num_of_nouns_requirements,nouns_vs_words_requirements,nouns_vs_length_requirements,num_of_nouns_benefits,nouns_vs_words_benefits,nouns_vs_length_benefits,num_of_verbs_company_profile,verbs_vs_words_company_profile,verbs_vs_length_company_profile,num_of_verbs_description,verbs_vs_words_description,verbs_vs_length_description,num_of_verbs_requirements,verbs_vs_words_requirements,verbs_vs_length_requirements,num_of_verbs_benefits,verbs_vs_words_benefits,verbs_vs_length_benefits,num_of_adjs_company_profile,adjs_vs_words_company_profile,adjs_vs_length_company_profile,num_of_adjs_description,adjs_vs_words_description,adjs_vs_length_description,num_of_adjs_requirements,adjs_vs_words_requirements,adjs_vs_length_requirements,num_of_adjs_benefits,adjs_vs_words_benefits,adjs_vs_length_benefits,num_of_propns_company_profile,propns_vs_words_company_profile,propns_vs_length_company_profile,num_of_propns_description,propns_vs_words_description,propns_vs_length_description,num_of_propns_requirements,propns_vs_words_requirements,propns_vs_length_requirements,num_of_propns_benefits,propns_vs_words_benefits,propns_vs_length_benefits,url_count,data_split
0,1,Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,unknown,0,1,0,Other,Internship,Bachelor's Degree,Information Technology and Services,Marketing,0,35000.0,50000.0,"['marketing', 'intern']","['PROPN', 'PROPN']","['food52', 'create', 'groundbreaking', 'award'...","['ADJ', 'VERB', 'NOUN', 'NOUN', 'VERB', 'NOUN'...","['food52', 'fast', 'grow', 'james', 'beard', '...","['NOUN', 'ADV', 'VERB', 'PROPN', 'PROPN', 'PRO...","['experience', 'content', 'management', 'syste...","['NOUN', 'NOUN', 'NOUN', 'NOUN', 'ADJ', 'CCONJ...",['unknown'],['ADJ'],US,NY,New York,0.9856,0.7345,0.0000,0.6486,"tensor([[[-0.2207, -0.1446, -0.2817, ..., 0....",0.022301,0.006270,[ 0.49690506 -0.13427249 0.5732839 0.082606...,28,0.345679,0.082111,43,0.518072,0.126100,39,0.527027,0.137809,0,0.000000,0.000000,16,0.197531,0.046921,17,0.204819,0.049853,14,0.189189,0.049470,0,0.000000,0.000000,8,0.098765,0.023460,5,0.060241,0.014663,16,0.216216,0.056537,1,1.000000,0.333333,27,0.333333,0.079179,15,0.180723,0.043988,3,0.040541,0.010601,0,0.000000,0.000000,0,Testing
1,2,Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,Bachelor's Degree,Marketing and Advertising,Customer Service,0,35000.0,50000.0,"['customer', 'service', 'cloud', 'video', 'pro...","['PROPN', 'PROPN', 'PROPN', 'PROPN', 'PROPN']","['90', 'seconds', 'world', 'cloud', 'video', '...","['NUM', 'PROPN', 'NOUN', 'PROPN', 'PROPN', 'PR...","['organise', 'focused', 'vibrant', 'awesome!do...","['VERB', 'ADJ', 'PROPN', 'PROPN', 'NOUN', 'NOU...","['expect', 'key', 'responsibility', 'communica...","['VERB', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'NUM',...","['usthrough', '90', 'seconds', 'team', 'gain',...","['ADP', 'NUM', 'PROPN', 'NOUN', 'VERB', 'NOUN'...",NZ,,Auckland

time: 66.5 ms (started: 2023-10-25 03:48:37 +00:00)


In [ ]:
y= df[[]]
X = df
# Split the data into training and testing sets with a fixed random_state
train, test, train_2, test_2 = train_test_split(X, y, test_size=0.2, random_state=42)

time: 39.9 ms (started: 2023-10-25 03:48:06 +00:00)


In [ ]:
country_df_fraudulent_train = train[train['fraudulent'] == 1].groupby('country')['fraudulent'].count().reset_index()
country_df_total_train = train.groupby('country')['job_id'].count().reset_index()
country_df_total_train = country_df_total_train.rename(columns={'job_id': 'total_number'})

country_df_fraudulent_test = test[test['fraudulent'] == 1].groupby('country')['fraudulent'].count().reset_index()
country_df_total_test = test.groupby('country')['job_id'].count().reset_index()
country_df_total_test = country_df_total_test.rename(columns={'job_id': 'total_number'})

time: 20.1 ms (started: 2023-10-25 03:51:55 +00:00)


In [ ]:
country_df_total_train

,country,total_number
0,AE,48
1,AL,1
2,AM,2
3,AR,9
4,AT,12
...,...,...
80,US,8557
81,VI,2
82,VN,4
83,ZA,32


time: 16.7 ms (started: 2023-10-25 03:51:59 +00:00)


In [ ]:
country_df_total_test

,country,total_number
0,AE,6
1,AT,2
2,AU,52
3,BE,20
4,BG,4
...,...,...
61,TW,3
62,UA,3
63,US,2099
64,VI,1


time: 19.4 ms (started: 2023-10-25 03:52:11 +00:00)


In [ ]:
country_df_fraudulent_train

,country,fraudulent
0,AE,1
1,AU,34
2,BH,5
3,BR,1
4,CA,12
5,EE,1
6,ES,1
7,GB,34
8,IN,2
9,MY,11


time: 13.8 ms (started: 2023-10-25 03:52:26 +00:00)


In [ ]:
country_df_fraudulent_test

,country,fraudulent
0,AU,6
1,EG,1
2,GB,8
3,ID,1
4,IN,2
5,MY,1
6,QA,1
7,TW,2
8,US,159


time: 40.2 ms (started: 2023-10-25 03:52:38 +00:00)


In [ ]:
country_df_ratio_train = pd.merge(country_df_total_train, country_df_fraudulent_train, on='country', how='left')
country_df_ratio_train['fraudulent'] = country_df_ratio_train['fraudulent'].fillna(0)
country_df_ratio_train

,country,total_number,fraudulent
0,AE,48,1.0
1,AL,1,0.0
2,AM,2,0.0
3,AR,9,0.0
4,AT,12,0.0
...,...,...,...
80,US,8557,571.0
81,VI,2,0.0
82,VN,4,0.0
83,ZA,32,1.0


time: 24.6 ms (started: 2023-10-25 03:53:08 +00:00)


In [ ]:
country_df_ratio_test = pd.merge(country_df_total_test, country_df_fraudulent_test, on='country', how='left')
country_df_ratio_test['fraudulent'] = country_df_ratio_test['fraudulent'].fillna(0)
country_df_ratio_test

,country,total_number,fraudulent
0,AE,6,0.0
1,AT,2,0.0
2,AU,52,6.0
3,BE,20,0.0
4,BG,4,0.0
...,...,...,...
61,TW,3,2.0
62,UA,3,0.0
63,US,2099,159.0
64,VI,1,0.0


time: 18.6 ms (started: 2023-10-25 03:57:10 +00:00)


In [ ]:
country_df_ratio_test['ratio'] = country_df_ratio_test['fraudulent'] / country_df_ratio_test['total_number']
country_df_ratio_test

,country,total_number,fraudulent,ratio
0,AE,6,0.0,0.000000
1,AT,2,0.0,0.000000
2,AU,52,6.0,0.115385
3,BE,20,0.0,0.000000
4,BG,4,0.0,0.000000
...,...,...,...,...
61,TW,3,2.0,0.666667
62,UA,3,0.0,0.000000
63,US,2099,159.0,0.075750
64,VI,1,0.0,0.000000


time: 24.5 ms (started: 2023-10-25 03:57:35 +00:00)


In [ ]:
country_df_ratio_train.to_csv("/content/drive/My Drive/BT4012 Group 06/Data/country_fraudulent_ratio_train.csv", index = False)

time: 10.2 ms (started: 2023-10-25 03:53:49 +00:00)


In [ ]:
country_df_ratio_test.to_csv("/content/drive/My Drive/BT4012 Group 06/Data/country_fraudulent_ratio_test.csv", index = False)

time: 14 ms (started: 2023-10-25 03:58:00 +00:00)


In [ ]:
country_df_ratio_train.set_index("country",inplace = True)

time: 1.76 ms (started: 2023-10-25 03:53:59 +00:00)


In [ ]:
country_df_ratio_test.set_index("country",inplace = True)

time: 1.66 ms (started: 2023-10-25 03:58:21 +00:00)


In [ ]:
country_df_ratio_train.loc['US']['ratio']

0.0667289938062405

time: 4.72 ms (started: 2023-10-25 03:54:05 +00:00)


In [ ]:
country_df_ratio_train.drop(['total_number', 'fraudulent'], axis = 1)

,ratio
country,
AE,0.020833
AL,0.000000
AM,0.000000
AR,0.000000
AT,0.000000
...,...
US,0.066729
VI,0.000000
VN,0.000000


time: 12.9 ms (started: 2023-10-25 03:54:12 +00:00)


In [ ]:
country_df_ratio_test.drop(['total_number', 'fraudulent'], axis = 1)

,ratio
country,
AE,0.000000
AT,0.000000
AU,0.115385
BE,0.000000
BG,0.000000
...,...
TW,0.666667
UA,0.000000
US,0.075750


time: 11.7 ms (started: 2023-10-25 03:58:43 +00:00)


In [ ]:
train_new = pd.merge(train, country_df_ratio_train, on='country', how='left')
train_new = train_new.rename(columns={'ratio': 'country_ratio','fraudulent_y' :'country_fraudulent','total_number':'country_total_number'})

time: 65.7 ms (started: 2023-10-25 04:08:32 +00:00)


In [ ]:
test_new = pd.merge(test, country_df_ratio_test, on='country', how='left')
test_new = test_new.rename(columns={'ratio': 'country_ratio','fraudulent_y' :'country_fraudulent','total_number':'country_total_number'})

time: 32.1 ms (started: 2023-10-25 04:08:35 +00:00)


In [ ]:
train_new

,job_id,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent_x,min_salary,max_salary,tokenized_title,pos_tags_title,tokenized_company_profile,pos_tags_company_profile,tokenized_description,pos_tags_description,tokenized_requirements,pos_tags_requirements,tokenized_benefits,pos_tags_benefits,country,province,city,sentiment score_profile,sentiment score_requirement,sentiment score_benefits,sentiment score_description,embeddings_profile,department_frequency,department_mean_target,mean_embeddings_description,num_of_nouns_company_profile,nouns_vs_words_company_profile,nouns_vs_length_company_profile,num_of_nouns_description,nouns_vs_words_description,nouns_vs_length_description,num_of_nouns_requirements,nouns_vs_words_requirements,nouns_vs_length_requirements,num_of_nouns_benefits,nouns_vs_words_benefits,nouns_vs_length_benefits,num_of_verbs_company_profile,verbs_vs_words_company_profile,verbs_vs_length_company_profile,num_of_verbs_description,verbs_vs_words_description,verbs_vs_length_description,num_of_verbs_requirements,verbs_vs_words_requirements,verbs_vs_length_requirements,num_of_verbs_benefits,verbs_vs_words_benefits,verbs_vs_length_benefits,num_of_adjs_company_profile,adjs_vs_words_company_profile,adjs_vs_length_company_profile,num_of_adjs_description,adjs_vs_words_description,adjs_vs_length_description,num_of_adjs_requirements,adjs_vs_words_requirements,adjs_vs_length_requirements,num_of_adjs_benefits,adjs_vs_words_benefits,adjs_vs_length_benefits,num_of_propns_company_profile,propns_vs_words_company_profile,propns_vs_length_company_profile,num_of_propns_description,propns_vs_words_description,propns_vs_length_description,num_of_propns_requirements,propns_vs_words_requirements,propns_vs_length_requirements,num_of_propns_benefits,propns_vs_words_benefits,propns_vs_length_benefits,url_count,country_total_number,country_fraudulent,country_ratio
0,2113,Operations,NaN,Want to build a 21st century financial service...,TransferWho?Co-founded by Skype’s first employ...,Is this you?Financial services or operations b...,Did someone say benefits?Apart from all-expens...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,0,35000.0,50000.0,"['operation', 'specialist']","['NOUN', 'NOUN']","['want', 'build', '21st', 'century', 'financia...","['VERB', 'VERB', 'ADJ', 'NOUN', 'ADJ', 'PROPN'...","['transferwho?co', 'found', 'skype', 'employee...","['PROPN', 'VERB', 'PROPN', 'NOUN', 'VERB', 'NO...","['you?financial', 'service', 'operation', 'bac...","['NOUN', 'NOUN', 'NOUN', 'NOUN', 'NUM', 'NOUN'...","['benefits?apart', 'expenses', 'pay', 'company...","['NOUN', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'DET'...",EE,37,Tallinn,0.9460,0.7579,0.8779,0.9728,"tensor([[[ 0.1128, -0.3043, -0.0584, ..., -0....",0.014961,0.000000,[ 0.46713018 -0.14550392 0.5929869 0.072091...,10,0.370370,0.095238,44,0.415094,0.104513,19,0.575758,0.152000,18,0.642857,0.165138,6,0.222222,0.057143,22,0.207547,0.052257,3,0.090909,0.024000,3,0.107143,0.027523,5,0.185185,0.047619,13,0.122642,0.030879,8,0.242424,0.064000,2,0.071429,0.018349,2,0.074074,0.019048,16,0.150943,0.038005,2,0.060606,0.016000,1,0.035714,0.009174,0,58,1.0,0.017241
1,7572,Sales,NaN,Pricing Recruiters is the leading global resou...,Working at Purolator International is more tha...,Responsible for supporting corporate profit ma...,Purolator International Provides Competitive B...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,0,35000.0,50000.0,"['purolator', 'international', 'pricing', 'fin...","['PROPN', 'PROPN', 'PROPN', 'PROPN', 'PROPN', ...","['pricing', 'recruiters', 'lead', 'global', 'r...","['PROPN', 'PROPN', 'VERB', 'ADJ', 'NOUN', 'NOU...","['work', 'purolator', 'international', 'job', ...","['VERB', 'PROPN', 'PROPN', 'NOUN', 'NOUN', 'VE..

time: 91.5 ms (started: 2023-10-25 04:08:40 +00:00)


In [ ]:
test_new

,job_id,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent_x,min_salary,max_salary,tokenized_title,pos_tags_title,tokenized_company_profile,pos_tags_company_profile,tokenized_description,pos_tags_description,tokenized_requirements,pos_tags_requirements,tokenized_benefits,pos_tags_benefits,country,province,city,sentiment score_profile,sentiment score_requirement,sentiment score_benefits,sentiment score_description,embeddings_profile,department_frequency,department_mean_target,mean_embeddings_description,num_of_nouns_company_profile,nouns_vs_words_company_profile,nouns_vs_length_company_profile,num_of_nouns_description,nouns_vs_words_description,nouns_vs_length_description,num_of_nouns_requirements,nouns_vs_words_requirements,nouns_vs_length_requirements,num_of_nouns_benefits,nouns_vs_words_benefits,nouns_vs_length_benefits,num_of_verbs_company_profile,verbs_vs_words_company_profile,verbs_vs_length_company_profile,num_of_verbs_description,verbs_vs_words_description,verbs_vs_length_description,num_of_verbs_requirements,verbs_vs_words_requirements,verbs_vs_length_requirements,num_of_verbs_benefits,verbs_vs_words_benefits,verbs_vs_length_benefits,num_of_adjs_company_profile,adjs_vs_words_company_profile,adjs_vs_length_company_profile,num_of_adjs_description,adjs_vs_words_description,adjs_vs_length_description,num_of_adjs_requirements,adjs_vs_words_requirements,adjs_vs_length_requirements,num_of_adjs_benefits,adjs_vs_words_benefits,adjs_vs_length_benefits,num_of_propns_company_profile,propns_vs_words_company_profile,propns_vs_length_company_profile,num_of_propns_description,propns_vs_words_description,propns_vs_length_description,num_of_propns_requirements,propns_vs_words_requirements,propns_vs_length_requirements,num_of_propns_benefits,propns_vs_words_benefits,propns_vs_length_benefits,url_count,country_total_number,country_fraudulent,country_ratio
0,4709,Sales,NaN,unknown,Stylect is a dynamic startup that helps helps ...,We don’t care where you studied or what your G...,We are negotiable on salary and there is the p...,0,1,0,Full-time,Entry level,Unspecified,Apparel & Fashion,Information Technology,0,35000.0,50000.0,"['python', 'engineer']","['PROPN', 'PROPN']",['unknown'],['ADJ'],"['stylect', 'dynamic', 'startup', 'help', 'hel...","['PROPN', 'ADJ', 'NOUN', 'VERB', 'VERB', 'NOUN...","['care', 'study', 'gpa', 'college', 'want', 's...","['VERB', 'VERB', 'PROPN', 'NOUN', 'VERB', 'ADJ...","['negotiable', 'salary', 'potential', 'equity'...","['ADJ', 'NOUN', 'NOUN', 'NOUN', 'ADJ', 'NOUN']",GB,,London,0.0000,0.0516,0.0000,0.9845,"tensor([[[-0.4404, 0.1208, -0.3744, ..., -0....",0.675685,0.044594,[ 0.50343186 -0.13808818 0.60929835 0.067156...,0,0.000000,0.000000,32,0.367816,0.093294,24,0.428571,0.105263,4,0.666667,0.181818,0,0.000000,0.000000,24,0.275862,0.069971,8,0.142857,0.035088,0,0.000000,0.000000,1,1.000000,0.333333,11,0.126437,0.032070,8,0.142857,0.035088,2,0.333333,0.090909,0,0.000000,0.000000,11,0.126437,0.032070,12,0.214286,0.052632,0,0.000000,0.000000,0,556,8.0,0.014388
1,11080,Sales,55000-75000,unknown,General Summary: Achieves maximum sales profit...,unknown,Great Health and DentalFast Advancement Opport...,1,0,0,Full-time,Entry level,High School or equivalent,Financial Services,Sales,0,55000.0,75000.0,"['entry', 'level', 'sale']","['NOUN', 'NOUN', 'NOUN']",['unknown'],['ADJ'],"['general', 'summary', 'achieve', 'maximum', '...","['PROPN', 'PROPN', 'VERB', 'ADJ', 'NOUN', 'NOU...",['unknown'],['ADJ'],"['great', 'health', 'dentalfast', 'advancement...","['PROPN', 'PROPN', 'PROPN', 'PROPN', 'PROPN', ...",US,OH,Cincinnati,0.0000,0.0000,0.8225,0.9880,"tensor([[[-0.4404, 0.1208, -0.3744, ..., -0....",0.675685,0.044594,[ 0.49963313 -0.11208937 0.5554975 0.089324...,0,0.000000,0.000000,129,0.608491,0.156364,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,36,0.169811,0.0436

time: 85.4 ms (started: 2023-10-25 04:08:52 +00:00)


In [ ]:
# Combine the encoded training and testing datasets
encoded_data = pd.concat([train_new, test_new]).sort_values(by = 'job_id')
encoded_data = encoded_data.reset_index(drop=True)

time: 200 ms (started: 2023-10-25 04:29:33 +00:00)


In [ ]:
encoded_data.to_csv('/content/drive/My Drive/BT4012 Group 06/Data/feature_engineering_data_v4.csv', index=False)

time: 6.44 s (started: 2023-10-25 04:29:35 +00:00)


In [ ]:
encoded_data = encoded_data.drop(['salary_range', 'tokenized_title', 'pos_tags_title', 'tokenized_company_profile', 'pos_tags_company_profile','tokenized_description','pos_tags_description', 'tokenized_requirements', 'pos_tags_requirements', 'tokenized_benefits', 'pos_tags_benefits', 'embeddings_profile', 'mean_embeddings_description'], axis = 1)


time: 9.1 ms (started: 2023-10-25 04:29:41 +00:00)


In [ ]:
encoded_data.to_csv('/content/drive/My Drive/BT4012 Group 06/Data/feature_engineering_data_v4.1.csv', index=False)

time: 2.94 s (started: 2023-10-25 04:29:41 +00:00)


In [ ]:
encoded_data

,job_id,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent_x,min_salary,max_salary,country,province,city,sentiment score_profile,sentiment score_requirement,sentiment score_benefits,sentiment score_description,department_frequency,department_mean_target,num_of_nouns_company_profile,nouns_vs_words_company_profile,nouns_vs_length_company_profile,num_of_nouns_description,nouns_vs_words_description,nouns_vs_length_description,num_of_nouns_requirements,nouns_vs_words_requirements,nouns_vs_length_requirements,num_of_nouns_benefits,nouns_vs_words_benefits,nouns_vs_length_benefits,num_of_verbs_company_profile,verbs_vs_words_company_profile,verbs_vs_length_company_profile,num_of_verbs_description,verbs_vs_words_description,verbs_vs_length_description,num_of_verbs_requirements,verbs_vs_words_requirements,verbs_vs_length_requirements,num_of_verbs_benefits,verbs_vs_words_benefits,verbs_vs_length_benefits,num_of_adjs_company_profile,adjs_vs_words_company_profile,adjs_vs_length_company_profile,num_of_adjs_description,adjs_vs_words_description,adjs_vs_length_description,num_of_adjs_requirements,adjs_vs_words_requirements,adjs_vs_length_requirements,num_of_adjs_benefits,adjs_vs_words_benefits,adjs_vs_length_benefits,num_of_propns_company_profile,propns_vs_words_company_profile,propns_vs_length_company_profile,num_of_propns_description,propns_vs_words_description,propns_vs_length_description,num_of_propns_requirements,propns_vs_words_requirements,propns_vs_length_requirements,num_of_propns_benefits,propns_vs_words_benefits,propns_vs_length_benefits,url_count,country_total_number,country_fraudulent,country_ratio
0,1,Marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,unknown,0,1,0,Other,Internship,Bachelor's Degree,Information Technology and Services,Marketing,0,35000.0,50000.0,US,NY,New York,0.9856,0.7345,0.0000,0.6486,0.022301,0.006270,28,0.345679,0.082111,43,0.518072,0.126100,39,0.527027,0.137809,0,0.000000,0.000000,16,0.197531,0.046921,17,0.204819,0.049853,14,0.189189,0.049470,0,0.000000,0.000000,8,0.098765,0.023460,5,0.060241,0.014663,16,0.216216,0.056537,1,1.000000,0.333333,27,0.333333,0.079179,15,0.180723,0.043988,3,0.040541,0.010601,0,0.000000,0.000000,0,2099,159.0,0.075750
1,2,Success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,Bachelor's Degree,Marketing and Advertising,Customer Service,0,35000.0,50000.0,NZ,,Auckland,0.9313,0.9929,0.9716,0.9951,0.000280,0.000000,42,0.392523,0.097222,75,0.367647,0.090689,62,0.473282,0.119461,47,0.456311,0.117207,17,0.158879,0.039352,35,0.171569,0.042322,24,0.183206,0.046243,23,0.223301,0.057357,10,0.093458,0.023148,25,0.122549,0.030230,19,0.145038,0.036609,16,0.155340,0.039900,22,0.205607,0.050926,42,0.205882,0.050786,5,0.038168,0.009634,7,0.067961,0.017456,0,267,0.0,0.000000
2,3,Sales,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,unknown,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,0,35000.0,50000.0,US,IA,Wever,0.9618,0.9260,0.0000,0.9509,0.675685,0.044594,30,0.361446,0.086957,13,0.448276,0.115044,66,0.550000,0.141026,0,0.000000,0.000000,10,0.120482,0.028986,5,0.172414,0.044248,32,0.266667,0.068376,0,0.000000,0.000000,13,0.156627,0.037681,6,0.206897,0.053097,15,0.125000,0.032051,1,1.000000,0.333333,22,0.265060,0.063768,4,0.137931,0.035398,5,0.041667,0.010684,0,0.000000,0.000000,0,8557,571.0,0.066729
3,4,Sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Re

time: 106 ms (started: 2023-10-25 04:27:56 +00:00)


In [ ]:
encoded_data.columns

Index(['job_id', 'department', 'company_profile', 'description',
       'requirements', 'benefits', 'telecommuting', 'has_company_logo',
       'has_questions', 'employment_type', 'required_experience',
       'required_education', 'industry', 'function', 'fraudulent_x',
       'min_salary', 'max_salary', 'country', 'province', 'city',
       'sentiment score_profile', 'sentiment score_requirement',
       'sentiment score_benefits', 'sentiment score_description',
       'department_frequency', 'department_mean_target',
       'num_of_nouns_company_profile', 'nouns_vs_words_company_profile',
       'nouns_vs_length_company_profile', 'num_of_nouns_description',
       'nouns_vs_words_description', 'nouns_vs_length_description',
       'num_of_nouns_requirements', 'nouns_vs_words_requirements',
       'nouns_vs_length_requirements', 'num_of_nouns_benefits',
       'nouns_vs_words_benefits', 'nouns_vs_length_benefits',
       'num_of_verbs_company_profile', 'verbs_vs_words_company_profile

time: 7.67 ms (started: 2023-10-25 04:19:45 +00:00)
